# Project: TMBD Movies Data analysis

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

<a id='intro'></a>
## Introduction

> **Tip**: In this section of the report, provide a brief introduction to the dataset you've selected for analysis. At the end of this section, describe the questions that you plan on exploring over the course of the report. Try to build your report around the analysis of at least one dependent variable and three independent variables.

> Votes for movies in the period 1960 to 2015.
* id - This is infact the movie_id as in the first dataset.
* imdb_id - A unique identifier for each movie in imdb dataset.
* cast - The name of lead and supporting actors.
* budget - The budget in which the movie was made.
* genre - The genre of the movie, Action, Comedy ,Thriller etc.
* homepage - A link to the homepage of the movie.
* keywords - The keywords or tags related to the movie.
* original_title - The title of the movie before translation or adaptation.
* overview - A brief description of the movie.
* popularity - A numeric quantity specifying the movie popularity.
* production_companies - The production house of the movie.
* release_date - The date on which it was released.
* release_year - The year on which it was released.
* revenue - The worldwide revenue generated by the movie.
* runtime - The running time of the movie in minutes.
* tagline - Movie's tagline.
* original_title - Title of the movie.
* vote_average - average ratings the movie recieved.
* vote_count - the count of votes recieved.
* budget_adj - The budget in which the movie was made adjusted for inflation.
* revenue_adj - The worldwide revenue generated by the movie adjusted for inflation.

This analysis will investigate the answer for the following questions:
* Q1: Which genres have the highest revenues?
* Q2: How the top 5 genres' revenue changed over time?
* Q3: Which genres are most popular? 
* Q4: How the top 5 popular genres changed over time?
* Q5: Who is the director whose movies got the highest revenues?

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools

# Limiting floats output to 2 decimal points and adding the thousands
# operator that places a comma between all thousands
pd.set_option('display.float_format', lambda x: '{:,.2f}'.format(x))

%matplotlib inline

<a id='wrangling'></a>
## Data Wrangling

> **Tip**: In this section of the report, you will load in the data, check for cleanliness, and then trim and clean your dataset for analysis. Make sure that you document your steps carefully and justify your cleaning decisions.

### General Properties

In [2]:
# Load the data
url = "https://drive.google.com/file/d/1UhOxP92bkolN9id5z6XziBE3MesIou_b/view?usp=sharing"
data = pd.read_csv(url)
#Check number of records and features.
data.shape

ParserError: Error tokenizing data. C error: Expected 277 fields in line 131, saw 434


In [ ]:
#Data at first glance
data.head(3).transpose()

Columns such as cast, keywords, genres, director and production_company have textual data separated by "|". It would be better to convert columns into list of strings rather than containing one long string.

#### Data types

In [ ]:
#check data types
data.dtypes

In [ ]:
#Data types summary
data.dtypes.value_counts()

We can notice that release_date is object data type rather than datetime. Also, release_year is of int type rather that datetime.

**Exploring columns with object data type**

Let's calculate summary statistics for object data type columns such as the number of unique values, the most common value and it's frequency.

In [ ]:
data[data.dtypes[data.dtypes == 'object'].index].describe().transpose()

**Exploring columns with numeric data types**

Get summary statistics for columns having numeric data types.

In [ ]:
data[data.dtypes[data.dtypes != 'object'].index].describe().transpose()

Check the distribution of values for columns with numeric data types.

In [ ]:
pd.plotting.hist_frame(data[data.dtypes[data.dtypes != 'object'].index], figsize= (12,10));

From the above summary statistics and distribution plots, we can notice the following:
* Popularity index is heavily skewed to the right such that 75% of movies or more have a popularity index less than or equal 0.71.
* 50% of movies or more have a budget and revenue values of zeros. Those values are most likely missing rather than zero.
* Minimum runtime is zero which doesn't make sense; it most likely missing. Also, maximum runtime is 900 which is inordinately long; it can be a series, documentary or just data entry mistake. We'll check.

#### Null values

We check which columns have null values?

In [ ]:
data.isnull().sum()

9 Columns have null values. All columns having null values are of object data type.

#### Duplicated rows

Check if there are any duplicated rows.

In [ ]:
print('Number of duplicated rows: ', data.duplicated().sum())

#### Outlires

From the summary statistics for numeric data above we noticed suspicious extreme values for **runtime** column. Let's first check maximum values.

In [ ]:
data[['original_title', 'genres', 'cast','release_year',
      'runtime']].sort_values('runtime', ascending= False).head(10)

As expected, movies with longest runtime are documentary and series. So, these values can't be considered as outliers.

### Data Cleaning 

We will do the following data cleaning steps:
1. Drop columns id, imbd_id, budget, tagline, runtime, keywords and overview since they are not important for our analysis.
2. Drop homepage column since most of its values are missing.
3. Replace zeros in revenue column by NA.
4. Drop rows with at least 1 missing values.
5. Drop duplicated row.
6. Convert data in cast, genres, director and production_companies columns to list of strings.
7. Convert data type for release_date column from "object" to "datetime".

In [ ]:
#Creat a copy of the original dataframe to work with.
data_cleaned = data.copy()

#### Step 1&2: Drop unnecessary columns.

In [ ]:
data_cleaned.drop(columns= [ 'id', 'budget','imdb_id', 'tagline',
                            'keywords', 'overview', 'runtime','homepage'],inplace= True)

#### Step 3: Replace zeros in revenue column.

We will replace the zeros in revenue column by NA since more than 50% of values in this column is missing. So, it would better dealing with these values as missing rather than zeros or any inaccurately imputed values.

In [ ]:
#Get index of movies with zero revenue and replace them by nan.
data_cleaned.loc[data_cleaned.revenue == 0, 'revenue'] = np.nan

#### Step 4: Drop rows with at least 1 missing value.

In [ ]:
data_cleaned.dropna(axis= 0, inplace= True)

#Check if there is any null value exists.
print('Number of null values: ', data_cleaned.isnull().any().sum())

#### Step 5: Drop the duplicated row.

In [ ]:
data_cleaned.drop_duplicates(inplace= True)

#Check if there's any duplcated row exists.
print('Number of duplicated rows: ', data_cleaned.duplicated().sum())

#### Step 6: Convert data into list of strings for columns (cast, genres, production_companies)

In [ ]:
columns = ['cast', 'genres', 'director','production_companies']
for col in columns:
    data_cleaned[col] = data_cleaned[col].apply(lambda x: x.split('|'))
    
#Check columns after conversion
data_cleaned[columns].head()

#### Step 7: Convert release_date column data type to datetime.

In [ ]:
data_cleaned['release_date'] = pd.to_datetime(data_cleaned['release_date'])

#Check data type of release_date column
data_cleaned[['release_date']].dtypes

<a id='eda'></a>
## Exploratory Data Analysis

### Q1: Which genres have the highest mean revenue?

To answer this question, we will a new dataframe in which the mean adjusted revenue is calculated for each genre. Then we plot the results on a bar chart.

In [ ]:
#Creat a new dataframe for genres, release_year and revenue adjusted.
genre_rev = data_cleaned[['genres', 'release_year', 'revenue_adj']].copy()
genre_rev.head()

In [ ]:
#Unpack lists of genres into separate rows and calculate the mean revenue for each genre.
df = genre_rev.explode('genres').groupby('genres')[['revenue_adj']].mean()\
                                    .sort_values(by= 'revenue_adj', ascending= False)

#Get the index of the top 5 genres in terms of mean revenue.
genre_top5_rev = df[:5].index.tolist()

#Plot genre Vs. Mean Revenue
df.plot(kind= 'bar', figsize= (10, 5))
plt.ylabel('Revenue')

We can see that top 5 genres in terms of mean revenue are:<br/>
 1. Animation.
 2. Adventure.
 3. Fantasy.
 4. Family.
 5. Science fiction.

### Q2: How the top 5 genres' revenue changed since 2000?

We build on our answer of the previous question and we will plot the mean revenue for each genre versus the year released.

In [ ]:
df2 = genre_rev.explode('genres').groupby(['release_year', 'genres']).mean()
df2.loc[(slice(None), genre_top5_rev), :].query('release_year >= 2000')\
                                .unstack().plot(figsize= (15,8), kind = 'line');
plt.ylabel('Mean Revenue');

Science fiction and adventure genres have an increasing trend in terms of mean revenue.

### Q3: Which genres are most popular? 

Similarly, we will create a new dataframe for the mean popularity for each genre.

In [ ]:
#Creat a new dataframe for genres, release_year and revenue adjusted.
genre_pop = data_cleaned[['genres', 'release_year', 'popularity']].copy()
genre_pop.head()

In [ ]:
#Unpack lists of genres into separate rows and calculate the mean popularity for each genre
df = genre_pop.explode('genres').groupby('genres')[['popularity']].mean()\
                                    .sort_values(by= 'popularity', ascending= False)
genre_top5_pop = df[:5].index.tolist()

#Plot genre Vs. Mean Popularity
df.plot(kind= 'bar', figsize= (10, 5))
plt.ylabel('Popularity');

The bar chart shows that the top 5 popular genres are:<br/>
 1. Adventure.
 2. Science fiction.
 3. Fantasy.
 4. Animation.
 5. Action.

In [ ]:
data_cleaned.plot(x= 'popularity', y= 'revenue_adj',kind= 'scatter');

The scatter plot above shows that popularity and revenue are positively correlated with some exceptions. There are some movie with very high revenue having relatively low popularity and vice versa.

### Q4: How the top 5 popular genres changed since 2000?

To answer this question we will plot the mean popularity for the 5 top popular genres since 2000.

In [ ]:
df3 = genre_pop.explode('genres').groupby(['release_year', 'genres']).mean()
df3.loc[(slice(None), genre_top5_pop), :].query('release_year >= 2000')\
                                .unstack().plot(figsize= (15,8), kind = 'line');
plt.ylabel('Mean Popularity');

The popularity of Science Fiction, Adventure and Action movies has grown significantly since 2000.

### Q5: Who is the director whose movies got the highest revenues?

We will create a new dataframe with the mean revenue for all movies directed by each director. Then we will plot the results on a bar chart.

In [ ]:
direct_rev = data_cleaned[['director', 'revenue_adj']].copy()
direct_rev.head()

In [ ]:
Top10_directors = direct_rev.explode('director').groupby('director').mean().sort_values('revenue_adj',
                                ascending= False)[:10]
Top10_directors.plot(kind='bar', figsize= (8,5));

The director of with the highest mean revenues is: Irwin Winkler. Let's check the movie genres directed by **Irwin Winkler**.

In [ ]:
#A stacked bar chart to represent the number of each genre.
data_cleaned[['director', 'genres']].explode('director').explode('genres').\
    query('director == "Irwin Winkler"').groupby('director')['genres'].value_counts().\
    unstack().plot(kind= 'bar', figsize= (6,5) ,stacked= True, rot= True);
plt.ylabel('Count')

Movies directed by **Irwin Winkler**:

In [ ]:
data_cleaned.explode('director').query('director == "Irwin Winkler"')

Only one movie is directed by **Irwin Winkler** called "The Net" released in 1995. According to the box office, the movie's revenues worldwide is $110.6 million which is the tenth of the revenue in our dataset. But we are not sure if this is a wrong number since the currency is not known for this dataset.

<a id='conclusions'></a>
## Conclusions

We can summarize our findings as follows:
* Adventure, fantasy, and science fiction movies are among the most popular and profitable genres.
* There's a positive correlation between popularity and revenue but causality can't be implied.
* There's a grown interest in adventure and science fiction movies since 2000.